# machine_learning_newton_ring
>牛顿环中心坐标估算主程序

## 00 基本依赖项加载

In [ ]:
# 加载依赖项
from cv2 import data
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from numpy import random
from numpy.lib.type_check import imag

import mindspore
import mindspore.dataset as ds

import mindspore.dataset.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C

from mindspore.common import dtype as mstype
from mindspore import context

from mindspore.common.initializer import TruncatedNormal
from mindspore import nn
from mindspore.train import Model
from mindspore.train.callback import Callback
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor, TimeMonitor
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore import Tensor
from mindspore.train.serialization import export
from mindspore.ops import operations as P

# 设置MindSpore的执行模式和设备
#context.GRAPH_MODE指向静态图模型，即在运行之前会把全部图建立编译完毕。设备指定为Ascend处理器。
context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")

## 01 数据生成与加载

In [ ]:
# 自定义项

# mindspore自定义数据集生成器
class DatasetGenerator:
    def __init__(self):
        img = np.array(Image.open("../data/img/train_img_1.jpg"))
        path = "../data/img/train_img_"
        for img_i in range(3049):
            img_nxt = np.array(Image.open(path+str(img_i+2)+".jpg"))
            if img_i == 0:
                img = np.stack((img,img_nxt))
            else:
                img_nxt = img_nxt[np.newaxis, :]
                img = np.vstack((img, img_nxt))
        self.data = img
        ct = np.load("../data/center/center.npy")
        self.label = ct

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)

In [ ]:
# 生成、加载./data中的数据
dataset_generator = DatasetGenerator()
dataset = ds.GeneratorDataset(dataset_generator, ["data", "label"], shuffle=False)

print("\n01 ---dataset generate complete---\n")

# 全部打印
# for data in dataset.create_dict_iterator():
#     print('{}'.format(data["data"]), '{}'.format(data["label"]))

# 创建数据集的迭代器p_data
p_data = dataset.create_dict_iterator()

## 02 数据处理

In [ ]:
# 数据处理

# 通道格式设置
hwc2chw_op = CV.HWC2CHW()
dataset = dataset.map(input_columns="data", operations=hwc2chw_op)
# 变成float32
type_cast_op = C.TypeCast(mstype.float32)
dataset = dataset.map(input_columns="data", operations=type_cast_op)
# 归一化
rescale = 1.0 / 255.5
shift = 0
rescale_op = CV.Rescale(rescale, shift)
dataset = dataset.map(input_columns="data", operations=rescale_op)
#打乱顺序
dataset = dataset.shuffle(buffer_size=3050)
#划分数据集
(ds_train,ds_test) = dataset.split([0.8,0.2])
#设置batch_size
batch_size = 50
ds_train = ds_train.batch(batch_size, drop_remainder=True)
ds_test  = ds_test.batch(batch_size, drop_remainder=True)

In [ ]:
print("测试集大小 ",ds_train.get_dataset_size()*batch_size,"组")
print("训练集大小 ",ds_test.get_dataset_size()*batch_size,"组")
# 选三组数据和标签打印，检查是否加载和处理正确
test_p_data = dataset.create_dict_iterator(output_numpy=True)
for i in range(3):
    test_data = next(test_p_data)
    num = random.randint(1,3050)
    # print('{}'.format(test_data["data"].shape), '{}'.format(test_data["label"])) #打印数据
    img = test_data["data"]
    ct = test_data["label"]
    print(img.shape, ct)
    plt.imshow(img, cmap="gray")
    plt.colorbar()
    plt.scatter(int(test_data["label"][0]), int(test_data["label"][1]), color='r',s=20)
    plt.show()  #打印图像
print("\n02 ---dataset proceed complete---\n")

## 03 模型构建

In [ ]:
# 定义模型
class VGG(nn.Cell): 
    # 神经网络的各层需要预先在__init__方法中定义，然后通过定义construct方法来完成神经网络的前向构造
    def __init__(self, num_class=2,channel=1,dropout_ratio=0.7,trun_sigma=0.01):
        super(VGG, self).__init__()
        self.num_class = num_class
        self.channel = channel
        self.dropout_ratio = dropout_ratio
        #设置卷积层
        self.conv11 = nn.Conv2d(self.channel, 64, #224*224*64
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv12 = nn.Conv2d(64, 64,            
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv21 = nn.Conv2d(64, 128, #112*112*128
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv22 = nn.Conv2d(128, 128,
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv31 = nn.Conv2d(128, 256, #56*56*256
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv32 = nn.Conv2d(256, 256,
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv33 = nn.Conv2d(256, 256,
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv41 = nn.Conv2d(256, 512, #28*28*512
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv42 = nn.Conv2d(512, 512,
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv43 = nn.Conv2d(512, 512,
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv51 = nn.Conv2d(512, 512, #14*14*512
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv52 = nn.Conv2d(512, 512,
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        self.conv53 = nn.Conv2d(512, 512,
                               kernel_size=3, stride=1, 
                               has_bias=True, pad_mode="same",
                               weight_init='xavier_uniform', bias_init='zeros')
        #设置最大池化层
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.max_pool_mid = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        #设置ReLU激活函数
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(self.dropout_ratio)
        
        self.fc1 = nn.Dense(7*7*512,4096,weight_init=TruncatedNormal(sigma=trun_sigma),bias_init = 0.1)
        self.fc2 = nn.Dense(4096, 4096, weight_init=TruncatedNormal(sigma=trun_sigma), bias_init=0.1)
        self.fc3 = nn.Dense(4096, 1024, weight_init=TruncatedNormal(sigma=trun_sigma), bias_init=0.1)
        self.fc4 = nn.Dense(1024, self.num_class, weight_init=TruncatedNormal(sigma=trun_sigma), bias_init=0.1)
    #构建模型    
    def construct(self, x):
        x = self.conv11(x)
        x = self.relu(x)
        x = self.conv12(x)          
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv21(x)
        x = self.relu(x)
        x = self.conv22(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv31(x)
        x = self.relu(x)
        x = self.conv32(x)
        x = self.relu(x)
        x = self.conv33(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv41(x)
        x = self.relu(x)
        x = self.conv42(x)
        x = self.relu(x)
        x = self.conv43(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv51(x)
        x = self.relu(x)
        x = self.conv52(x)
        x = self.relu(x)
        x = self.conv53(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return x
# 构建完成后，可以使用print将神经网络中的各层参数全部打印出来
print(VGG())
print("\n03-1 ---model set complete---\n")

In [ ]:
# 定义损失函数（Loss）和优化器（Optimizer）

net=VGG(num_class=2, channel=1, dropout_ratio=0.5,trun_sigma=0.01)
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
#opt
fc_weight_params = list(filter(lambda x: 'fc' in x.name and 'weight' in x.name, net.trainable_params()))
other_params=list(filter(lambda x: 'fc' not in x.name or 'weight' not in x.name, net.trainable_params()))
group_params = [{'params': fc_weight_params, 'weight_decay': 0.01},
                {'params': other_params},
                {'order_params': net.trainable_params()}]
net_opt = nn.Adam(group_params, learning_rate=0.0001, weight_decay=0.0)
print("\n03-2 ---loss and optimizer set complete---\n")

## 04 开始训练

In [ ]:
# 定义回调函数收集模型的损失值和精度值，为之后对训练过程进行可视化显示做准备
class StepLossAccInfo(Callback):
    def __init__(self, model, eval_dataset, steps_loss, steps_eval):
        self.model = model
        self.eval_dataset = eval_dataset
        self.steps_loss = steps_loss
        self.steps_eval = steps_eval

    def step_end(self, run_context):
        cb_params = run_context.original_args()
        cur_epoch = cb_params.cur_epoch_num
        cur_step = (cur_epoch-1)*2400 + cb_params.cur_step_num
        self.steps_loss["loss_value"].append(str(cb_params.net_outputs))
        self.steps_loss["step"].append(str(cur_step))
        if cur_step % 48 == 0:
            acc = self.model.eval(self.eval_dataset, dataset_sink_mode=True)
            self.steps_eval["step"].append(cur_step)
            self.steps_eval["acc"].append(acc["acc"])
            
model = Model(net, loss_fn=net_loss, optimizer=net_opt, metrics={'acc', 'loss'})

time_cb = TimeMonitor()    # 监控每个epoch训练的时间
loss_cb = LossMonitor(per_print_times=ds_train.get_dataset_size())  #LossMonitor用于打印损失函数

config_ck = CheckpointConfig(save_checkpoint_steps=960,keep_checkpoint_max=10)
ckpoint_cb = ModelCheckpoint(prefix="VGG", directory="./output_model/checkpoint", config=config_ck)

steps_loss = {"step": [], "loss_value": []}
steps_eval = {"step": [], "acc": []}
step_loss_acc_info = StepLossAccInfo(model, ds_test, steps_loss, steps_eval)

print("============== Starting Training ==============")
epoch_size = 500 # 训练次数
model.train(epoch_size, ds_train, callbacks=[loss_cb, ckpoint_cb, time_cb, step_loss_acc_info], dataset_sink_mode=True)
print('Checkpoints after resuming training:')
print('\n'.join(sorted([x for x in os.listdir("./output_model/checkpoint") if x.startswith('VGG')])))
print("\n04 ---training complete---\n")